# Main ERP Pipeline

This notebook contains processing steps for BCI III P300 dataset.
The goal is to classify ERP forms for target and common stimuli using peak features

Description of the pipeline can be found there: http://asi.insa-rouen.fr/enseignants/~arakoto/publi/rakoto_bci.pdf

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import sys
sys.path.append("../src")

In [13]:
from load import load_bci_iii_data as load_data

In [14]:
from preprocess import process_signal, restore_raw, get_events

In [15]:
import pandas as pd

In [16]:
import matplotlib.pyplot as plt

In [25]:
import numpy as np

In [26]:
from sklearn.model_selection import GridSearchCV, train_test_split, ShuffleSplit, TimeSeriesSplit, PredefinedSplit

In [63]:
from selection import cross_validate, select_estimators

In [65]:
from sklearn.metrics import accuracy_score

In [66]:
from selection import predict_characters

In [59]:
from models import create_svc_pipeline

# Load and preprocess data

You can find the description of each method in the related src/preprocess.py file

### Train trials preprocessing

In [18]:
train_subject = "Subject_A_Train"

In [19]:
sessions_df, true_characters = load_data(train_subject)

In [ ]:
epochs = process_signal(sessions_df)

### Test trials preprocessing

In [21]:
test_subject = "Subject_A_Test"

In [22]:
test_sessions_df, test_true_characters = load_data(test_subject)

In [ ]:
test_epochs = process_signal(test_sessions_df)

# Train model

You can find the description of each method in the related src/models.py and src/selections.py file

In [60]:
# Number of estimators
n_estimators = 17

# Number of flashing sequences per estimator
# n_sequences = 12

# Number of train characters per estimator
n_characters = 5

# Regualization coefficients for SVM model
C_values = [0.01, 0.05, 0.1, 0.5, 1]

In [ ]:
hyperparameters = cross_validate(sessions_df, create_svc_pipeline, C_values, n_estimators, n_characters)

In [64]:
estimators = select_estimators(hyperparameters)

# Multiclass accuracy check

You can find the description of each method in the related src/selections.py file

In [ ]:
You can find the description of each method in the related src/preprocess.py file

In [74]:
n_test_sequences = 15 # 15 or 5, as it was for competition

In [ ]:
predicted_characters = predict_characters(estimators, test_epochs, test_sessions_df, test_true_characters, n_test_sequences)

In [78]:
accuracy_score([c for c in test_true_characters], predicted_characters)

0.96